<a href="https://colab.research.google.com/github/martinkenas/Review_Analysis/blob/main/Vartotoj%C5%B3_atsiliepim%C5%B3_analiz%C4%97s_variklio_suk%C5%ABrimas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prijungiam Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importuojam būtinus modulius
import pandas as pd
import re

# Nustatomas kelias iki failo Google Drive
# Čia reikia pakeisti 'kelias/iki/jūsų/failo.csv' į jūsų failo vietą Google Drive
file_path = '/content/drive/My Drive/IMDB.csv'

# Skaitymas duomenų failo
df = pd.read_csv(file_path)

# Rodomos pirmosios eilutės, kad patikrintume, ar duomenys įkelti sėkmingai
print(df.head())

# Teksto valymo funkcija
def clean_text(text):
    text = re.sub(r'<.*?>', ' ', text)  # pašalinami HTML tagai
    text = re.sub(r'\d+', ' ', text)  # pašalinami skaičiai
    text = re.sub(r'\W+', ' ', text)  # pašalinami specialieji simboliai
    text = text.lower()  # visos raidės paverčiamos mažosiomis
    text = re.sub(r'\s+', ' ', text)  # pašalinami nereikalingi tarpai
    return text.strip()

# Taikome valymo funkciją 'review' stulpeliui ir sukuriame naują stulpelį 'cleaned_review'
df['cleaned_review'] = df['review'].apply(clean_text)

# Atspausdiname pavyzdžius, kad pamatytume pradinį ir išvalytą tekstą
print(df[['review', 'cleaned_review']].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  one of the other reviewers has mentioned that ...  
1  a wonderful little production the filming t

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sukurkite TF-IDF vektorizatorių
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['cleaned_review']).toarray()

# Konvertuokite sentiment stulpelį į skaitines reikšmes
y = df['sentiment'].map({'positive': 1, 'negative': 0}).values


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Daliname duomenis į mokymo ir testavimo rinkinius
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistinė regresija
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(f'Logistinės regresijos tikslumas: {accuracy_score(y_test, y_pred_log_reg)}')

# Atsitiktiniai miškai
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(f'Atsitiktinių miškų tikslumas: {accuracy_score(y_test, y_pred_rf)}')

# Dirbtinis neuroninis tinklas
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print(f'Dirbtinio neuroninio tinklo tikslumas: {accuracy_score(y_test, y_pred_mlp)}')


Logistinės regresijos tikslumas: 0.8693
Atsitiktinių miškų tikslumas: 0.8396
Dirbtinio neuroninio tinklo tikslumas: 0.8635


In [4]:
# Pavyzdys, identifikuojant atsiliepimus, į kuriuos reikia sureaguoti, naudojant logistinės regresijos modelį
y_proba_log_reg = log_reg.predict_proba(X_test)

# Identifikuojame atsiliepimus, kuriuose yra labai neigiamas sentimentas (tikimybė > 0.9)
critical_reviews = [i for i, proba in enumerate(y_proba_log_reg) if proba[0] > 0.9]

# Identifikuojame atsiliepimus, kuriuose yra tam tikrų raktinių žodžių
keywords = ['terrible', 'awful', 'worst', 'horrible']  # Čia galite įvesti savo raktinius žodžius
keyword_reviews = [i for i, review in enumerate(df['cleaned_review']) if any(keyword in review for keyword in keywords)]

# Identifikuojame ilgus atsiliepimus (pvz., daugiau nei 1000 simbolių)
long_reviews = [i for i, review in enumerate(df['cleaned_review']) if len(review) > 1000]

# Surandame visus atsiliepimus, į kuriuos reikia sureaguoti
reviews_to_respond = set(critical_reviews + keyword_reviews + long_reviews)

print(f'Atsiliepimų, į kuriuos reikia sureaguoti, indeksai: {reviews_to_respond}')

# Atspausdiname pirmus tris neigiamus atsiliepimus, į kuriuos reikia sureaguoti
negative_reviews = [index for index in sorted(reviews_to_respond) if df['sentiment'][index] == 'negative']
for i, review_index in enumerate(negative_reviews):
    if i >= 3:  # Jei atspausdinome pirmus tris atsiliepimus, nutraukiame ciklą
        break
    print(f'Neigiamas atsiliepimas {i+1}:')
    print(df['review'][review_index])
    print('\n')


Atsiliepimų, į kuriuos reikia sureaguoti, indeksai: {0, 2, 4, 7, 8, 10, 12, 13, 14, 17, 19, 20, 21, 23, 24, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 48, 50, 51, 54, 55, 56, 58, 59, 60, 65, 66, 67, 68, 69, 70, 71, 72, 74, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 88, 89, 90, 92, 93, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 116, 117, 119, 122, 126, 128, 130, 131, 133, 135, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 149, 151, 152, 153, 154, 155, 156, 158, 161, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 185, 186, 187, 188, 189, 190, 191, 194, 195, 196, 197, 198, 200, 201, 202, 207, 210, 211, 214, 215, 216, 217, 218, 221, 222, 223, 224, 225, 226, 227, 228, 229, 232, 238, 239, 241, 242, 246, 247, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 267, 270, 272, 273, 274, 275, 276, 277, 278, 280, 282, 284, 285, 286, 288, 293, 295, 296, 297, 298

In [5]:
from sklearn.metrics import recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

# Apsibrėžiame funkciją modelių vertinimui
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model_name = model.__class__.__name__
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"{model_name} tikslumas: {accuracy:.4f}, atpažinimo tikslumas: {recall:.4f}, F1 balas: {f1:.4f}")

    # Kryžminė patikra
    cross_val = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"{model_name} vidutinis kryžminės patikros tikslumas: {cross_val.mean():.4f}\n")

    return accuracy, recall, f1

# Įvertiname kiekvieną modelį
log_reg_metrics = evaluate_model(log_reg, X_train, y_train, X_test, y_test)
rf_clf_metrics = evaluate_model(rf_clf, X_train, y_train, X_test, y_test)
mlp_clf_metrics = evaluate_model(mlp_clf, X_train, y_train, X_test, y_test)

# Hiperparametrų derinimas RandomForest modeliui
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30]
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"Geriausiai parinkti hiperparametrai: {grid_search.best_params_}")
print(f"Geriausias kryžminės patikros tikslumas: {grid_search.best_score_:.4f}")

# Sukurkime geriausią modelį pagal parinktus hiperparametrus
best_rf = grid_search.best_estimator_
best_rf_metrics = evaluate_model(best_rf, X_train, y_train, X_test, y_test)


LogisticRegression tikslumas: 0.8693, atpažinimo tikslumas: 0.8768, F1 balas: 0.8711
LogisticRegression vidutinis kryžminės patikros tikslumas: 0.8647

RandomForestClassifier tikslumas: 0.8396, atpažinimo tikslumas: 0.8373, F1 balas: 0.8403
RandomForestClassifier vidutinis kryžminės patikros tikslumas: 0.8299

MLPClassifier tikslumas: 0.8635, atpažinimo tikslumas: 0.8710, F1 balas: 0.8654
MLPClassifier vidutinis kryžminės patikros tikslumas: 0.8584

Geriausiai parinkti hiperparametrai: {'max_depth': None, 'n_estimators': 100}
Geriausias kryžminės patikros tikslumas: 0.8312
RandomForestClassifier tikslumas: 0.8371, atpažinimo tikslumas: 0.8341, F1 balas: 0.8377
RandomForestClassifier vidutinis kryžminės patikros tikslumas: 0.8303

